In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")

In [6]:
import psycopg2

load_dotenv()
dbconn = os.getenv("DBCONN")

# establish database connection using connection string from .env
conn = psycopg2.connect(dbconn)

# create a cursor
cur = conn.cursor()



In [3]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS btc_data(
        id INTEGER PRIMARY KEY,
        title TEXT,
        author TEXT,
        published_at TIMESTAMP
    );
''')

In [4]:
conn.commit()

In [ ]:
import psycopg2
import pandas as pd
from psycopg2.extras import execute_values

# Load (or reuse your existing df)
df = pd.read_csv("btc_data.csv", parse_dates=["Dates.N"])

# --- keep ONLY the columns you insert ---
df_ins = df[["ID", "Title", "Author", "Dates.N"]].copy()

# --- clean types ---
df_ins["ID"] = pd.to_numeric(df_ins["ID"], errors="coerce")               # -> float/int or NaN
df_ins["Title"] = df_ins["Title"].astype(str)
df_ins["Author"] = df_ins["Author"].astype(str)
df_ins["Dates.N"] = pd.to_datetime(df_ins["Dates.N"], errors="coerce")    # invalid -> NaT

# --- drop rows that can't be inserted (NULL id or timestamp) ---
df_ins = df_ins.dropna(subset=["ID", "Dates.N"])

# --- build rows in the correct order & python types ---
rows = [
    (int(r.ID), r.Title, r.Author, pd.Timestamp(r["Dates.N"]).to_pydatetime())
    for _, r in df_ins.iterrows()
]

conn = psycopg2.connect(dsn=dbconn)
cur = conn.cursor()

execute_values(
    cur,
    """
    INSERT INTO btc_data (id, title, author, published_at)
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET
        title = EXCLUDED.title,
        author = EXCLUDED.author,
        published_at = EXCLUDED.published_at;
    """,
    rows
)

conn.commit()
cur.close()
conn.close()

✅ Inserted 58 rows.
